## Human or Bot Detection Using Bidirectional LSTM


In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('preprocessed_filename (1).csv')

In [3]:
df.head()

,Texts,content,corpus
0,I buy this iPhone last 2 weeks.From my experi...,human,buy iphon last week experi iphon good perform ...
1,The impact of renewable energy sources on the ...,bot,impact renew energi sourc mine industri natur ...
2,You don't have anything to complaint when you...,human,anyth complaint get iphon big billion day sale...
3,Awesome experience nice product from apple.ca...,human,awesom experi nice product appl camera experi ...
4,Super fantastic Apple iPhone...I repent not t...,human,super fantast appl iphon repent earlier beast ...


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop(['content', 'Texts'],axis=1)

In [6]:
## Get the Dependent features
y=df['content']

In [14]:
y.value_counts()

human    272
bot      249
Name: Texts2, dtype: int64

In [7]:
X.shape

(1530, 1)

In [8]:
y.shape

(1530,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.12.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [12]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [13]:
import nltk
import re


In [14]:
df['corpus']

0       buy iphon last week experi iphon good perform ...
1       impact renew energi sourc mine industri natur ...
2       anyth complaint get iphon big billion day sale...
3       awesom experi nice product appl camera experi ...
4       super fantast appl iphon repent earlier beast ...
                              ...                        
1525    qualiti quit good satisfiedbtw iphon phone inc...
1526    use artifici intellig market advertis present ...
1527    first iphon love perform camera user interfac ...
1528    impact digit technolog creativ innov constantl...
1529    impact digit technolog journal media constantl...
Name: corpus, Length: 1530, dtype: object

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in df['corpus']] 
onehot_repr

[[2442,
  4929,
  1608,
  2025,
  2134,
  4929,
  4697,
  2318,
  1074,
  903,
  2159,
  3506,
  461,
  3702,
  2016,
  1018,
  903,
  2893,
  1964,
  3539,
  2134,
  2954,
  2060,
  4349,
  1929,
  3276,
  1929,
  3009,
  389,
  3455,
  1029],
 [3657,
  933,
  2139,
  3996,
  316,
  3495,
  2639,
  2391,
  4440,
  3822,
  1833,
  3140,
  4791,
  257,
  3696,
  3045,
  1616],
 [315, 2640, 2562, 4929, 1732, 1522, 4873, 3076, 3632, 2783, 4929, 419, 1433],
 [4907,
  2134,
  2890,
  183,
  3009,
  3348,
  2134,
  4907,
  2167,
  903,
  4210,
  4911,
  3506,
  1161,
  2167,
  2318,
  2442,
  4929,
  2841,
  2518,
  814],
 [1161,
  2393,
  3009,
  4929,
  1200,
  4449,
  4275,
  3706,
  3903,
  1058,
  2953,
  1161,
  3348,
  4736,
  1608,
  903,
  1335,
  4742,
  655,
  2421,
  4929,
  3675,
  4213,
  1143],
 [163,
  4929,
  2454,
  3632,
  1319,
  4845,
  606,
  2893,
  2635,
  4535,
  4929,
  979,
  80,
  4929,
  2130,
  4929,
  389,
  2684,
  829],
 [1319,
  849,
  1325,
  183,
  4545,
 

In [17]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[3506  461 3702 ...  389 3455 1029]
 [   0    0    0 ... 3696 3045 1616]
 [   0    0    0 ... 4929  419 1433]
 ...
 [   0    0    0 ... 1000 2567 4015]
 [   0    0    0 ... 4568 2472  183]
 [   0    0    0 ... 2480  183 4923]]


In [18]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
len(embedded_docs),y.shape

(1530, (1530,))

In [20]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
le = LabelEncoder()

# Fit the LabelEncoder to the categorical data and transform it into numerical labels
y = le.fit_transform(y)

# Print the numerical labels
print(y)

[1 0 1 ... 1 0 0]


In [21]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [22]:
X_final.shape,y_final.shape

((1530, 20), (1530,))

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [24]:
X_train

array([[   0,    0,    0, ..., 4463, 4697, 2607],
       [   0,    0,    0, ...,   45,  720, 1695],
       [   0,    0,    0, ...,   45,  720, 1695],
       ...,
       [   0,    0,    0, ...,  773, 1359, 3743],
       [   0,    0,    0, ..., 3999, 4121, 4791],
       [   0,    0,    0, ..., 3941, 2472, 1775]])

In [25]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

### Model Training

In [26]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
17/17 [==============================] - 5s 89ms/step - loss: 0.6435 - accuracy: 0.5600 - val_loss: 0.4591 - val_accuracy: 0.8356
Epoch 2/100
17/17 [==============================] - 1s 51ms/step - loss: 0.2436 - accuracy: 0.9220 - val_loss: 0.2201 - val_accuracy: 0.9426
Epoch 3/100
17/17 [==============================] - 1s 52ms/step - loss: 0.0700 - accuracy: 0.9824 - val_loss: 0.0897 - val_accuracy: 0.9683
Epoch 4/100
17/17 [==============================] - 1s 50ms/step - loss: 0.0218 - accuracy: 0.9961 - val_loss: 0.0754 - val_accuracy: 0.9743
Epoch 5/100
17/17 [==============================] - 1s 49ms/step - loss: 0.0060 - accuracy: 0.9980 - val_loss: 0.0669 - val_accuracy: 0.9822
Epoch 6/100
17/17 [==============================] - 1s 48ms/step - loss: 0.0022 - accuracy: 0.9990 - val_loss: 0.0899 - val_accuracy: 0.9762
Epoch 7/100
17/17 [==============================] - 1s 47ms/step - loss: 8.9097e-04 - accuracy: 1.0000 - val_loss: 0.1057 - val_accuracy: 0.9762
Ep

Epoch 57/100
17/17 [==============================] - 1s 49ms/step - loss: 3.2000e-05 - accuracy: 1.0000 - val_loss: 0.1283 - val_accuracy: 0.9782
Epoch 58/100
17/17 [==============================] - 1s 49ms/step - loss: 3.0215e-05 - accuracy: 1.0000 - val_loss: 0.1266 - val_accuracy: 0.9782
Epoch 59/100
17/17 [==============================] - 1s 48ms/step - loss: 2.7290e-05 - accuracy: 1.0000 - val_loss: 0.1266 - val_accuracy: 0.9782
Epoch 60/100
17/17 [==============================] - 1s 48ms/step - loss: 2.6111e-05 - accuracy: 1.0000 - val_loss: 0.1270 - val_accuracy: 0.9782
Epoch 61/100
17/17 [==============================] - 1s 49ms/step - loss: 2.8593e-05 - accuracy: 1.0000 - val_loss: 0.1284 - val_accuracy: 0.9782
Epoch 62/100
17/17 [==============================] - 1s 49ms/step - loss: 2.3886e-05 - accuracy: 1.0000 - val_loss: 0.1285 - val_accuracy: 0.9782
Epoch 63/100
17/17 [==============================] - 1s 53ms/step - loss: 2.5386e-05 - accuracy: 1.0000 - val_loss: 0

### Performance Metrics And Accuracy

In [27]:

y_pred1=model1.predict(X_test)

16/16 [==============================] - 1s 8ms/step


In [28]:
y_pred1

array([[9.99985039e-01],
       [1.52291150e-07],
       [9.99986649e-01],
       [1.46305922e-07],
       [1.51711461e-07],
       [1.92729502e-07],
       [1.81426131e-07],
       [9.99989688e-01],
       [9.99997318e-01],
       [1.68431782e-07],
       [2.41492472e-07],
       [9.99989390e-01],
       [2.36532216e-07],
       [1.57403903e-07],
       [1.51555284e-07],
       [1.47270995e-07],
       [9.99991894e-01],
       [7.68388247e-07],
       [1.73149260e-07],
       [1.47935978e-07],
       [9.99987483e-01],
       [9.99988258e-01],
       [5.44513114e-07],
       [1.68284217e-07],
       [1.83590953e-06],
       [9.99991596e-01],
       [1.88107165e-07],
       [1.12079806e-05],
       [9.99651432e-01],
       [1.50864679e-07],
       [2.25133945e-07],
       [1.06929667e-06],
       [9.99983311e-01],
       [1.51440560e-07],
       [1.55052874e-07],
       [1.84478537e-07],
       [1.88181630e-07],
       [9.99986768e-01],
       [9.99994695e-01],
       [9.99991000e-01],


In [29]:
def process(x):
    if(x>=0.5):
        return 1
    return 0

In [30]:
import numpy as np
y_pred1 = np.vectorize(process)(y_pred1)

In [31]:
len(X_test)

505

In [32]:
y_pred1

array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
    

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(y_test,y_pred1)

array([[214,   8],
       [  3, 280]], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9782178217821782

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       222
           1       0.97      0.99      0.98       283

    accuracy                           0.98       505
   macro avg       0.98      0.98      0.98       505
weighted avg       0.98      0.98      0.98       505



In [37]:
import pickle

# save the model to a file using pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model1, f)


In [38]:

# load the model from the file
with open('model.pkl', 'rb') as f:
    clf = pickle.load(f)

# make predictions on new data

y_pred = clf.predict(X_test)


16/16 [==============================] - 1s 7ms/step


In [39]:
y_pred

array([[9.99985039e-01],
       [1.52291150e-07],
       [9.99986649e-01],
       [1.46305922e-07],
       [1.51711461e-07],
       [1.92729502e-07],
       [1.81426131e-07],
       [9.99989688e-01],
       [9.99997318e-01],
       [1.68431782e-07],
       [2.41492472e-07],
       [9.99989390e-01],
       [2.36532216e-07],
       [1.57403903e-07],
       [1.51555284e-07],
       [1.47270995e-07],
       [9.99991894e-01],
       [7.68388247e-07],
       [1.73149260e-07],
       [1.47935978e-07],
       [9.99987483e-01],
       [9.99988258e-01],
       [5.44513114e-07],
       [1.68284217e-07],
       [1.83590953e-06],
       [9.99991596e-01],
       [1.88107165e-07],
       [1.12079806e-05],
       [9.99651432e-01],
       [1.50864679e-07],
       [2.25133945e-07],
       [1.06929667e-06],
       [9.99983311e-01],
       [1.51440560e-07],
       [1.55052874e-07],
       [1.84478537e-07],
       [1.88181630e-07],
       [9.99986768e-01],
       [9.99994695e-01],
       [9.99991000e-01],


In [40]:
def process1(x):
    if(x>=0.5):
        return "human"
    return "bot"

In [41]:
import numpy as np
y_pred1 = np.vectorize(process1)(y_pred)

In [42]:
y_pred1[0:5]

array([['human'],
       ['bot'],
       ['human'],
       ['bot'],
       ['bot']], dtype='<U5')

In [43]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  163, 1309, 2963, 4856])

In [44]:
y_test[0:5]

array([1, 0, 1, 0, 0])

In [45]:
corpus ='use regular techniqu regular techniqu l l regular help prevent overfit improv gener abil model'

In [46]:
onehot_repr=[one_hot(corpus,voc_size)] 
onehot_repr

[[1319,
  2437,
  2410,
  2437,
  2410,
  1244,
  1244,
  2437,
  2283,
  2881,
  4435,
  4210,
  646,
  4434,
  587]]

In [47]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0    0    0 1319 2437 2410 2437 2410 1244 1244 2437 2283
  2881 4435 4210  646 4434  587]]


In [48]:
X_final=np.array(embedded_docs)

In [49]:
y_pred = clf.predict(X_final)

1/1 [==============================] - 0s 21ms/step


In [50]:
y_pred

array([[2.8318408e-07]], dtype=float32)

In [51]:
process1(y_pred)

'bot'

In [52]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('texts_data.csv')

# Load the second CSV file
df2 = pd.read_csv('myexcel.csv')

# Merge the two dataframes based on the 'Texts' and 'content' columns
merged_df = pd.merge(df1, df2, on=['Texts', 'content'])

# Write the merged dataframe to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
